<a href="https://colab.research.google.com/github/jwang44/crispy-fiesta/blob/main/grid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook implements the grid search for feature and model parameter together

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

Mounted at /content/drive
/content/drive/MyDrive


## Load the data and get basic features

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('./train.csv',engine='python')
test = pd.read_csv('./test.csv',engine='python')

In [ ]:
X_train = train.body  # train texts
y_train = train.subreddit # train subreddits
X_test = test.body  # test texts

In [ ]:
from sklearn.preprocessing import Normalizer, LabelEncoder
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [ ]:
# transform target labels to values
le = LabelEncoder()
y_train_num = le.fit_transform(y_train.values) # convert category from string to numerical (!!!!! update the variables in kcross fold)

# vectorize word count
vectorizer = CountVectorizer()
vectors_train = vectorizer.fit_transform(X_train)
vectors_test = vectorizer.transform(X_test)

normalizer_train = Normalizer()
vectors_train= normalizer_train.transform(vectors_train)
vectors_test= normalizer_train.transform(vectors_test)

# print(vectorizer.get_feature_names())
print(vectors_train.shape)
print(vectors_test.shape)

(1999, 15365)
(1378, 15365)


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# put it all together: remove stop words and punctuation, tfidf, lemmatization, normalization
stop_words = text.ENGLISH_STOP_WORDS

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

class New_LemmaTokenizer:
     def __init__(self):
       self.wnl = WordNetLemmatizer()
     def __call__(self, doc):
       return [self.wnl.lemmatize(t,pos =get_wordnet_pos(t)) for t in word_tokenize(doc) if t.isalpha()]

tf_idf_transformer = TfidfTransformer()
vectorizer = CountVectorizer(stop_words = stop_words, tokenizer = New_LemmaTokenizer(), ngram_range=(1, 2)) #unigram+bigram:ngram_range=(1, 2), only bigram:ngram_range=(2, 2)
vectors_train_stop_Lemma = vectorizer.fit_transform(X_train)
vectors_train_stop_tfidf_Lemma = tf_idf_transformer.fit_transform(vectors_train_stop_Lemma)
vectors_test_stop_Lemma = vectorizer.transform(X_test)
vectors_test_stop_tfidf_Lemma = tf_idf_transformer.transform(vectors_test_stop_Lemma)
vectors_train_stop_tfidf_Lemma = normalizer_train.transform(vectors_train_stop_tfidf_Lemma)
vectors_test_stop_tfidf_Lemma = normalizer_train.transform(vectors_test_stop_tfidf_Lemma)
normalizer_l1 = Normalizer(norm='l1')
vectors_train_stop_tfidf_l1_Lemma = normalizer_l1.transform(vectors_train_stop_tfidf_Lemma)
vectors_test_stop_tfidf_l1_Lemma = normalizer_l1.transform(vectors_test_stop_tfidf_Lemma)

#print(vectorizer.get_feature_names())
print(vectors_train_stop_tfidf_Lemma.shape)
#print(vectors_test_stop_tfidf_Lemma.shape)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far', 'make', 'u'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(1999, 70414)


## Grid search

In [ ]:
from sklearn.feature_selection import SelectKBest, SelectPercentile, chi2, mutual_info_classif, f_classif, SelectFpr, SelectFwe, SelectFdr, RFE, RFECV, SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

from sklearn.model_selection import KFold, cross_val_score

### N-gram

In [ ]:
# grid search for the N-gram range 
pipeline = Pipeline([
    ('vect', CountVectorizer(stop_words = stop_words)),
    ('tfidf', TfidfTransformer()),
    ('normalize',Normalizer()),
    ('select', RFECV(estimator=LinearSVC(),step=2800))
])

parameters = {  
    'vect__ngram_range': ((1, 1), (1, 2), (1, 3), (2, 2), (3, 3))
}
gs_model = GridSearchCV(pipeline, parameters, cv=10, n_jobs=-1)
gs_model = gs_model.fit(X_train, y_train_num)
print(gs_model.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_model.best_params_[param_name]))

0.9474773869346734
vect__ngram_range: (1, 2)


### linearSVC without lemmatization

In [ ]:
# grid search for linearSVC without lemmatization
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('normalize',Normalizer()),
    ('select', SelectPercentile()),
    ('clf', LinearSVC()),
])

parameters = {  
    'vect__ngram_range': ((1, 1), (1, 2)),  # use unigram or unigram+bigram
    'vect__stop_words':(None, text.ENGLISH_STOP_WORDS), # whether to remove stopwords
    'tfidf__use_idf': (True, False), #whether to use idf
    'normalize__norm': ('l1','l2'),  #L1 or L2 normalization
    'select__percentile': (20, 40, 60, 80, 100),  # selected percentile of the original features
    'select__score_func': (chi2, f_classif,mutual_info_classif), # scoring function type
    'clf__C': (0.01, 0.1, 1, 10, 100) # Regularization parameter
}
gs_model = GridSearchCV(pipeline, parameters, cv=10, n_jobs=-1)
gs_model = gs_model.fit(X_train, y_train_num)
print(gs_model.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_model.best_params_[param_name]))

0.9514824120603015
clf__C: 10
normalize__norm: 'l2'
select__percentile: 100
select__score_func: <function chi2 at 0x7f54485fc320>
tfidf__use_idf: True
vect__ngram_range: (1, 2)
vect__stop_words: frozenset({'yourself', 'ltd', 'afterwards', 'only', 'already', 'everyone', 'move', 'by', 'themselves', 'someone', 'detail', 'hereafter', 'becomes', 'down', 'then', 'rather', 'thin', 'though', 'ie', 'over', 'latter', 'sometime', 'on', 'un', 'with', 'ten', 'through', 'meanwhile', 'no', 'whom', 'becoming', 'most', 'sixty', 'very', 'beforehand', 'whence', 'will', 'because', 'thereby', 'go', 'below', 'were', 'an', 'whereby', 'much', 're', 'whoever', 'always', 'sometimes', 'found', 'has', 'some', 'she', 'see', 'during', 'without', 'but', 'de', 'last', 'top', 'these', 'everything', 'others', 'mostly', 'there', 'may', 'could', 'put', 'how', 'those', 'onto', 'except', 'four', 'anyway', 'among', 'be', 'its', 'cry', 'enough', 'whatever', 'thence', 'else', 'which', 'none', 'himself', 'never', 'moreover', '

### linearSVC with lemmatization

In [ ]:
# grid search for linearSVC with lemmatization
pipeline = Pipeline([
    ('tfidf', TfidfTransformer()),
    ('normalize',Normalizer()),
    ('select', SelectPercentile()),
    ('clf', LinearSVC()),
])

parameters = {
    'tfidf__use_idf': (True, False),#whether to use idf
    'normalize__norm': ('l1','l2'),#L1 or L2 normalization
    'select__percentile': (20, 40, 60, 80, 100), # selected percentile of the original features
    'select__score_func': (chi2, f_classif，mutual_info_classif)# scoring function type
    'clf__C': (0.01, 0.1, 1, 10, 100) # Regularization parameter
}
gs_model = GridSearchCV(pipeline, parameters, cv=10, n_jobs=-1)
gs_model = gs_model.fit(vectors_train_stop_Lemma, y_train_num)
print(gs_model.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_model.best_params_[param_name]))

0.940969849246231
clf__C: 1
normalize__norm: 'l2'
select__percentile: 40
select__score_func: <function chi2 at 0x7fa8ddef2290>
tfidf__use_idf: True


### RBF SVC with lemmatization

In [ ]:
# grid search for RBFSVC with lemmatization
pipeline = Pipeline([
    ('tfidf', TfidfTransformer()),
    ('normalize',Normalizer()),
    ('select', SelectPercentile()),
    ('clf', SVC()),
])

parameters = {
    'tfidf__use_idf': (True, False),#whether to use idf
    'normalize__norm': ('l1','l2'),#L1 or L2 normalization
    'select__percentile': (20, 40, 60, 80, 100), # selected percentile of the original features
    'select__score_func': (chi2, f_classif，mutual_info_classif)# scoring function type
    'clf__C': (0.01, 0.1, 1, 10, 100) # Regularization parameter
}
gs_model = GridSearchCV(pipeline, parameters, cv=10, n_jobs=-1)
gs_model = gs_model.fit(vectors_train_stop_tfidf_Lemma, y_train_num)
print(gs_model.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_model.best_params_[param_name]))

0.9319748743718591
clf__C: 10
normalize__norm: 'l2'
select__percentile: 100
select__score_func: <function chi2 at 0x7f27ac7b0320>
tfidf__use_idf: True


### MultinomialNB with lemmatization

In [ ]:
# grid search for MultinomialNB with lemmatization
pipeline = Pipeline([
    ('tfidf', TfidfTransformer()),
    ('normalize',Normalizer()),
    ('select', SelectPercentile()),
    ('clf', MultinomialNB())
])

parameters = {   
    'tfidf__use_idf': (True, False),#whether to use idf
    'normalize__norm': ('l1','l2'),#L1 or L2 normalization
    'select__percentile': (20, 40, 60, 80, 100), # selected percentile of the original features
    'select__score_func': (chi2, f_classif，mutual_info_classif)# scoring function type
    'clf__alpha': (1e-10, 1e-5, 0.1, 0.5, 1, 2) # smoothing parameter
}
gs_model = GridSearchCV(pipeline, parameters, cv=10, n_jobs=-1)
gs_model = gs_model.fit(vectors_train_stop_Lemma, y_train_num)
print(gs_model.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_model.best_params_[param_name]))

0.9309723618090452
clf__alpha: 0.1
normalize__norm: 'l2'
select__percentile: 60
select__score_func: <function chi2 at 0x7fa8ddef2290>
tfidf__use_idf: False


### BernoulliNB with lemmatization

In [ ]:
# grid search for BernoulliNB with lemmatization
pipeline = Pipeline([
    ('tfidf', TfidfTransformer()),
    ('normalize',Normalizer()),
    ('select', SelectPercentile()),
    ('clf', BernoulliNB())
])

parameters = {   
    'tfidf__use_idf': (True, False),#whether to use idf
    'normalize__norm': ('l1','l2'),#L1 or L2 normalization
    'select__percentile': (20, 40, 60, 80, 100), # selected percentile of the original features
    'select__score_func': (chi2, f_classif，mutual_info_classif)# scoring function type
    'clf__alpha': (1e-10, 1e-5, 0.1, 0.5, 1, 2) # smoothing parameter
}
gs_model = GridSearchCV(pipeline, parameters, cv=10, n_jobs=-1)
gs_model = gs_model.fit(vectors_train_stop_Lemma, y_train_num)
print(gs_model.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_model.best_params_[param_name]))

0.9094623115577889
clf__alpha: 1e-05
normalize__norm: 'l2'
select__percentile: 60
select__score_func: <function chi2 at 0x7fa8ddef2290>
tfidf__use_idf: True
